# Matriz Origen Destino

En el notebook anterior calculamos las trayectorias de los usuarios de Twitter. En este notebook vamos a obtener la matriz Origen-Destino a partir de dichas trayectorias.

In [1]:
%load_ext autoreload
%autoreload 2
import geopandas as gpd
from shapely.geometry import LineString, Point
from datetime import datetime, timedelta
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from IPython.display import display, HTML

Leemos las trayectorias que calculamos

In [2]:
trayectorias = gpd.read_file("../output/trayectorias.gpkg")
trayectorias.geometry.crs = "EPSG:32614" # TODO: Resolver en 00-preproceso!!
trayectorias['dia'] = pd.to_datetime(trayectorias['dia'], format='%Y-%m-%dT%H:%M:%S')
trayectorias['intervalo'] = pd.to_datetime(trayectorias['intervalo'], format='%Y-%m-%dT%H:%M:%S')
trayectorias.set_index(['dia', 'intervalo', 'uname'], inplace=True)
trayectorias.head()

separation  \
dia        intervalo           uname                          
2014-10-29 2014-10-29 10:00:00 AlejandroSordo   9237.812888   
                               GreenPistachio    669.068739   
                               MariinePa         738.825055   
                               PeRrOiMpOstOr   14826.892697   
           2014-10-29 14:00:00 23Tannie         4788.381689   

                                                                                        geometry  
dia        intervalo           uname                                                              
2014-10-29 2014-10-29 10:00:00 AlejandroSordo  LINESTRING (471865.220 2140838.487, 479166.515...  
                               GreenPistachio  LINESTRING (525419.137 2180731.374, 525190.231...  
                               MariinePa       LINESTRING (484571.596 2132370.418, 485244.847...  
                               PeRrOiMpOstOr   LINESTRING (490924.227 2134725.016, 481962.962...  
           2014-10-29 14:00:00 23Tannie        LINESTRING (483439.703 2144613.319, 485053.985...

Y las zonas de análisis

In [3]:
zonas = gpd.read_file("../data/DistritosEODHogaresZMVM2017.shp")
zonas.head()

,Distrito,Descripcio,geometry
0,001,Centro HistÃ³rico,"POLYGON ((485707.699 2149605.446, 485792.518 2..."
1,002,Buenavista-Reforma,"POLYGON ((485002.129 2152136.266, 485069.252 2..."
2,003,Tlatelolco,"POLYGON ((486206.963 2152150.606, 486612.741 2..."
3,004,Morelos,"POLYGON ((488510.758 2150946.712, 488607.167 2..."
4,005,"Moctezuma, Terminal de Autobuses de Oriente (T...","POLYGON ((489445.177 2150604.177, 489535.192 2..."


## Un dia y un intervalo

Primero vamos a calcular la matriz para un sólo dia y un intervalo. 

In [35]:
prueba = trayectorias.loc['2014-11-26', '2014-11-26 06:00',:].copy()
prueba.head()

separation  \
dia        intervalo           uname                         
2014-11-26 2014-11-26 06:00:00 APaulinaCM     13630.270452   
                               ARMAND_SOSA    12326.778546   
                               ASTRIDDIXIE    10496.000395   
                               AleeMijaangos   5086.649280   
                               Alewarhol      19502.382121   

                                                                                       geometry  
dia        intervalo           uname                                                             
2014-11-26 2014-11-26 06:00:00 APaulinaCM     LINESTRING (492666.157 2145640.730, 492659.230...  
                               ARMAND_SOSA    LINESTRING (484135.756 2132479.009, 483626.976...  
                               ASTRIDDIXIE    LINESTRING (488482.463 2141106.261, 482065.330...  
                               AleeMijaangos  LINESTRING (485333.721 2135830.200, 482768.977...  
                               Alewarhol      LINESTRING (485525.936 2179115.244, 480706.258...

Para cada usuario tenemos que encontrar los puntos de origen y destino de las trayectorias, para eso podemos crear dos dataframes diferentes, una para los orígenes y otra para los destinos.

In [50]:
origen = prueba['geometry'].apply(lambda l: Point(l.coords[0])).reset_index()
origen.crs = "EPSG:32614"
destino = prueba['geometry'].apply(lambda l: Point(l.coords[-1])).reset_index()
destino.crs = "EPSG:32614"

Y asignamos las zonas a cada uno de ellos 

In [51]:
origen = gpd.sjoin(origen.reset_index(), zonas).drop(['index', 'index_right', 'Descripcio'], axis=1)
destino = gpd.sjoin(destino.reset_index(), zonas).drop(['index','index_right', 'Descripcio'], axis=1)
origen.head()

,dia,intervalo,uname,geometry,Distrito
0,2014-11-26,2014-11-26 06:00:00,APaulinaCM,POINT (492666.157 2145640.730),035
6,2014-11-26,2014-11-26 06:00:00,AmandaPatrizi16,POINT (493000.139 2148084.367),035
171,2014-11-26,2014-11-26 06:00:00,nayma0,POINT (493694.128 2145000.106),035
1,2014-11-26,2014-11-26 06:00:00,ARMAND_SOSA,POINT (484135.756 2132479.009),068
78,2014-11-26,2014-11-26 06:00:00,PacoLopezGalvez,POINT (482701.897 2131614.354),068


Realmente sólo necesitamos conservar los índices temporales (día e intervalo), el `uname` y los distritos de origen y destino. Cada viaje está identificado por el `uname`

In [56]:
origen_destino = origen[['dia', 'intervalo', 'uname' , 'Distrito']].merge(destino[['uname' , 'Distrito']], on='uname')
origen_destino.rename({'Distrito_x':'distrito_origen', 'Distrito_y':'distrito_destino'}, axis=1, inplace=True)
origen_destino.head()

,dia,intervalo,uname,distrito_origen,distrito_destino
0,2014-11-26,2014-11-26 06:00:00,APaulinaCM,035,057
1,2014-11-26,2014-11-26 06:00:00,AmandaPatrizi16,035,036
2,2014-11-26,2014-11-26 06:00:00,nayma0,035,036
3,2014-11-26,2014-11-26 06:00:00,ARMAND_SOSA,068,010
4,2014-11-26,2014-11-26 06:00:00,PacoLopezGalvez,068,001


Ya sólo necesitamos agregar por origen y destino y contar la cantidad de viajes en cada grupo

In [58]:
origen_destino = origen_destino.groupby(['distrito_origen', 'distrito_destino']).count()
origen_destino.head(20)

dia  intervalo  uname
distrito_origen distrito_destino                       
001             058                 1          1      1
                070                 1          1      1
002             001                 2          2      2
                002                 5          5      5
                008                 1          1      1
                015                 2          2      2
                059                 1          1      1
                110                 1          1      1
003             002                 1          1      1
005             002                 1          1      1
006             008                 1          1      1
                016                 1          1      1
                058                 1          1      1
                064                 1          1      1
007             008                 1          1      1
                016                 1          1      1
008             001                 1          1      1
                002                 1          1      1
                008                 3          3      3
                010                 1          1      1

Esto que hicimos fua sólo para un dia y un intervalo, ahora hay que hacerlo para todos. En el `DataFrame` resulante tenemos que tener, para cada dia e intervalo, un resultado igual al que tenemos arriba en `origen_destino`

## Para todos los dias

Lo único que necesitamos hacer para obtener el resultado que buuscamos es definir una fución de agregación que regrese, para cada dia e intervalo, un `DataFrame` como el que tenemos arriba

In [4]:
def get_od(df, zonas):    
    origen = df['geometry'].apply(lambda l: Point(l.coords[0])).reset_index()
    origen.crs = "EPSG:32614"
    destino = df['geometry'].apply(lambda l: Point(l.coords[-1])).reset_index()
    destino.crs = "EPSG:32614"
    origen = gpd.sjoin(origen.reset_index(), zonas).drop(['index', 'index_right', 'Descripcio'], axis=1)
    destino = gpd.sjoin(destino.reset_index(), zonas).drop(['index','index_right', 'Descripcio'], axis=1)
    origen_destino = origen[['dia', 'intervalo', 'uname' , 'Distrito']].merge(destino[['uname' , 'Distrito']], on='uname')
    origen_destino.rename({'Distrito_x':'distrito_origen', 'Distrito_y':'distrito_destino'}, axis=1, inplace=True)
    origen_destino = origen_destino.groupby(['distrito_origen', 'distrito_destino']).count()
    return origen_destino

od_final = trayectorias.groupby(level=['dia', 'intervalo']).apply(get_od, zonas=zonas)

Vamos a seleccionar el mismo dia e intervalos que usamos arriba y comparemos

In [74]:
od_final.loc['2014-11-26', '2014-11-26 06:00',:].head(20)

dia  \
dia        intervalo           distrito_origen distrito_destino        
2014-11-26 2014-11-26 06:00:00 001             058                 1   
                                               070                 1   
                               002             001                 2   
                                               002                 5   
                                               008                 1   
                                               015                 2   
                                               059                 1   
                                               110                 1   
                               003             002                 1   
                               005             002                 1   
                               006             008                 1   
                                               016                 1   
                                               058                 1   
                                               064                 1   
                               007             008                 1   
                                               016                 1   
                               008             001                 1   
                                               002                 1   
                                               008                 3   
                                               010                 1   

                                                                 intervalo  \
dia        intervalo           distrito_origen distrito_destino              
2014-11-26 2014-11-26 06:00:00 001             058                       1   
                                               070                       1   
                               002             001                       2   
                                               002                       5   
                                               008                       1   
                                               015                       2   
                                               059                       1   
                                               110                       1   
                               003             002                       1   
                               005             002                       1   
                               006             008                       1   
                                               016                       1   
                                               058                       1   
                                               064                       1   
                               007             008                       1   
                                               016                       1   
                               008             001                       1   
                                               002                       1   
                                               008                       3   
                                               010                       1   

                                                                 uname  
dia        intervalo           distrito_origen distrito_destino         
2014-11-26 2014-11-26 06:00:00 001             058                   1  
                                               070                   1  
                               002             001                   2  
                                               002                   5  
                                               008                   1  
                                               015                   2  
                                               059                   1  
                                               110                   1  
                               00

In [67]:
origen_destino.head(20)

dia  intervalo  uname
distrito_origen distrito_destino                       
001             058                 1          1      1
                070                 1          1      1
002             001                 2          2      2
                002                 5          5      5
                008                 1          1      1
                015                 2          2      2
                059                 1          1      1
                110                 1          1      1
003             002                 1          1      1
005             002                 1          1      1
006             008                 1          1      1
                016                 1          1      1
                058                 1          1      1
                064                 1          1      1
007             008                 1          1      1
                016                 1          1      1
008             001                 1          1      1
                002                 1          1      1
                008                 3          3      3
                010                 1          1      1

Perfecto! Son iguales.
Arreglemos los nombres de las columnas, en realidad todas son iguales y representan el número de viajes.

In [5]:
od_final = od_final[['dia']]
od_final.rename({'dia':'viajes'}, axis=1, inplace=True)
od_final.head()

viajes
dia        intervalo           distrito_origen distrito_destino        
2014-10-29 2014-10-29 10:00:00 043             001                    1
                               044             068                    1
                               059             058                    1
                               171             171                    1
           2014-10-29 14:00:00 001             001                   10

## Agregados por semana

Para cada dia las muestras son relativamente pequeñas, entonces vamos a sacar primero los agregados para cada semana y luego obtener el promedio final. Recordemos que aquí tenemos todos los dias del periodo muestreado y que, en principio vamos a tener dos tipos dew comportamiento diferentes: dias laborales y fines de semana. Entonces, primero, vamos a obtener la matriz sólo para los días laborales. Antes de eso tenemos que ajustar el índice de intervalo porque incluye el dia, entonces no podemos agrupar bien

In [6]:
od_final = od_final.reset_index()
od_final['intervalo_time'] = od_final['intervalo'].apply(lambda x: x.strftime('%H:%M:%S'))
od_final = (od_final.drop(['intervalo'], axis=1)
            .rename({'intervalo_time':'intervalo'}, axis=1)
            .set_index(['dia', 'intervalo', 'distrito_origen'])
           )
od_final

distrito_destino  viajes
dia        intervalo distrito_origen                         
2014-10-29 10:00:00  043                          001       1
                     044                          068       1
                     059                          058       1
                     171                          171       1
           14:00:00  001                          001      10
...                                               ...     ...
2015-04-10 07:00:00  170                          170       1
                     171                          171       1
                     178                          207       1
                     191                          191       1
                     197                          197       1

[115750 rows x 2 columns]

Seleccionemos los dias laborales

In [7]:
od_laborales = od_final.loc[od_final.index.get_level_values('dia').weekday.isin(range(0,5)),:,:]

Agrupemos estos datos por semana 

In [8]:
laborales_semana = od_final.groupby([pd.Grouper(level='dia', freq='W'), 'intervalo', 'distrito_origen', 'distrito_destino']).sum()
laborales_semana

viajes
dia        intervalo distrito_origen distrito_destino        
2014-11-02 02:00:00  001             001                    4
                                     002                    2
                                     010                    1
                                     017                    1
                     002             002                    3
...                                                       ...
2015-04-12 23:00:00  174             174                    2
                     188             186                    1
                     192             192                    1
                     195             195                    1
                     207             207                    2

[79374 rows x 1 columns]

Finalmente, calculemos los estadísticos (suma, media, desviación estándar) para cada uno de los intervalos sobre todo el periodo de estudio.

In [9]:
promedios_intervalo = laborales_semana.groupby(['intervalo', 'distrito_origen', 'distrito_destino']).agg(['sum', 'mean', 'std'])
promedios_intervalo

viajes                    
                                              sum      mean       std
intervalo distrito_origen distrito_destino                           
02:00:00  001             001                  45  2.250000  1.371707
                          002                  24  1.411765  0.507300
                          003                   3  1.000000  0.000000
                          004                   1  1.000000       NaN
                          006                   2  1.000000  0.000000
...                                           ...       ...       ...
23:00:00  174             174                   2  2.000000       NaN
          188             186                   1  1.000000       NaN
          192             192                   1  1.000000       NaN
          195             195                   1  1.000000       NaN
          207             207                   2  2.000000       NaN

[26801 rows x 3 columns]

Exportamos el resultado como csv

In [15]:
promedios_intervalo.reset_index(col_level=1).to_csv("../output/matriz_od_semana.csv")

In [16]:
import hvplot.pandas

ModuleNotFoundError: No module named 'hvplot'